In [2]:
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from IPython.core.interactiveshell import InteractiveShell

import os
import warnings
import numpy as np
import pandas as pd

In [3]:
warnings.filterwarnings(action='ignore')
InteractiveShell.ast_node_interactivity = "all"

In [4]:
input_dir = '../Database/'
file = 'train.csv'
data = pd.read_csv(os.path.join(input_dir, file))
data

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [5]:
label_encoder=LabelEncoder()
encoder_data = label_encoder.fit_transform(list(data['대출기간']))
data['대출기간']=pd.DataFrame(encoder_data)
data

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,0,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,1,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,0,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,1,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,0,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,0,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [6]:
encoder_data = label_encoder.fit_transform(list(data['근로기간']))
data['근로기간']=pd.DataFrame(encoder_data)
data

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,9,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,2,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,8,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,0,11,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,1,15,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,0,2,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,1,2,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,0,0,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,0,8,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [7]:
one_hot_encoded= pd.get_dummies(list(data['주택소유상태']))
set(data['주택소유상태'])

{'ANY', 'MORTGAGE', 'OWN', 'RENT'}

In [8]:
data.pop('주택소유상태')
data = pd.concat([data.iloc[:, :4], one_hot_encoded, data.iloc[:, 4:]], axis=1)
data

0            RENT
1        MORTGAGE
2        MORTGAGE
3        MORTGAGE
4            RENT
           ...   
96289    MORTGAGE
96290    MORTGAGE
96291    MORTGAGE
96292    MORTGAGE
96293        RENT
Name: 주택소유상태, Length: 96294, dtype: object

,ID,대출금액,대출기간,근로기간,ANY,MORTGAGE,OWN,RENT,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,9,False,False,False,True,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,2,False,True,False,False,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,8,False,True,False,False,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,0,11,False,True,False,False,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,1,15,False,False,False,True,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,0,2,False,True,False,False,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,1,2,False,True,False,False,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,0,0,False,True,False,False,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,0,8,False,True,False,False,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [9]:
one_hot_encoded= pd.get_dummies(list(data['대출목적']))
set(data['대출목적'])

{'기타',
 '부채 통합',
 '소규모 사업',
 '신용 카드',
 '의료',
 '이사',
 '자동차',
 '재생 에너지',
 '주요 구매',
 '주택',
 '주택 개선',
 '휴가'}

In [10]:
data.pop('대출목적')
data = pd.concat([data.iloc[:, :11], one_hot_encoded, data.iloc[:, 11:]], axis=1)
data

0        부채 통합
1        주택 개선
2        부채 통합
3        부채 통합
4        주요 구매
         ...  
96289    신용 카드
96290    주택 개선
96291    신용 카드
96292    부채 통합
96293    신용 카드
Name: 대출목적, Length: 96294, dtype: object

,ID,대출금액,대출기간,근로기간,ANY,MORTGAGE,OWN,RENT,연간소득,부채_대비_소득_비율,...,주요 구매,주택,주택 개선,휴가,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,9,False,False,False,True,72000000,18.90,...,False,False,False,False,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,2,False,True,False,False,130800000,22.33,...,False,False,True,False,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,8,False,True,False,False,96000000,8.60,...,False,False,False,False,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,0,11,False,True,False,False,132000000,15.09,...,False,False,False,False,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,1,15,False,False,False,True,71736000,25.39,...,True,False,False,False,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,0,2,False,True,False,False,210000000,9.33,...,False,False,False,False,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,1,2,False,True,False,False,132000000,5.16,...,False,False,True,False,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,0,0,False,True,False,False,84000000,11.24,...,False,False,False,False,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,0,8,False,True,False,False,66330000,17.30,...,False,False,False,False,2,1378368,818076.0,0.0,0.0,D


In [11]:
encoder_data = label_encoder.fit_transform(list(data['대출등급']))
data['대출등급']=pd.DataFrame(encoder_data)
data

,ID,대출금액,대출기간,근로기간,ANY,MORTGAGE,OWN,RENT,연간소득,부채_대비_소득_비율,...,주요 구매,주택,주택 개선,휴가,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,9,False,False,False,True,72000000,18.90,...,False,False,False,False,0,0,0.0,0.0,0.0,2
1,TRAIN_00001,14400000,1,2,False,True,False,False,130800000,22.33,...,False,False,True,False,0,373572,234060.0,0.0,0.0,1
2,TRAIN_00002,12000000,0,8,False,True,False,False,96000000,8.60,...,False,False,False,False,0,928644,151944.0,0.0,0.0,0
3,TRAIN_00003,14400000,0,11,False,True,False,False,132000000,15.09,...,False,False,False,False,0,325824,153108.0,0.0,0.0,2
4,TRAIN_00004,18000000,1,15,False,False,False,True,71736000,25.39,...,True,False,False,False,0,228540,148956.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,0,2,False,True,False,False,210000000,9.33,...,False,False,False,False,0,974580,492168.0,0.0,0.0,2
96290,TRAIN_96290,28800000,1,2,False,True,False,False,132000000,5.16,...,False,False,True,False,0,583728,855084.0,0.0,0.0,4
96291,TRAIN_96291,14400000,0,0,False,True,False,False,84000000,11.24,...,False,False,False,False,0,1489128,241236.0,0.0,0.0,0
96292,TRAIN_96292,15600000,0,8,False,True,False,False,66330000,17.30,...,False,False,False,False,2,1378368,818076.0,0.0,0.0,3


In [14]:
data.set_index(data['ID'], inplace=True)

In [15]:
data

,ID,대출금액,대출기간,근로기간,ANY,MORTGAGE,OWN,RENT,연간소득,부채_대비_소득_비율,...,주요 구매,주택,주택 개선,휴가,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
ID,,,,,,,,,,,,,,,,,,,,,
TRAIN_00000,TRAIN_00000,12480000,0,9,False,False,False,True,72000000,18.90,...,False,False,False,False,0,0,0.0,0.0,0.0,2
TRAIN_00001,TRAIN_00001,14400000,1,2,False,True,False,False,130800000,22.33,...,False,False,True,False,0,373572,234060.0,0.0,0.0,1
TRAIN_00002,TRAIN_00002,12000000,0,8,False,True,False,False,96000000,8.60,...,False,False,False,False,0,928644,151944.0,0.0,0.0,0
TRAIN_00003,TRAIN_00003,14400000,0,11,False,True,False,False,132000000,15.09,...,False,False,False,False,0,325824,153108.0,0.0,0.0,2
TRAIN_00004,TRAIN_00004,18000000,1,15,False,False,False,True,71736000,25.39,...,True,False,False,False,0,228540,148956.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_96289,TRAIN_96289,14400000,0,2,False,True,False,False,210000000,9.33,...,False,False,False,False,0,974580,492168.0,0.0,0.0,2
TRAIN_96290,TRAIN_96290,28800000,1,2,False,True,False,False,132000000,5.16,...,False,False,True,False,0,583728,855084.0,0.0,0.0,4
TRAIN_96291,TRAIN_96291,14400000,0,0,False,True,False,False,84000000,11.24,...,False,False,False,False,0,1489128,241236.0,0.0,0.0,0


In [ ]:
def generate_PCA_data(data: pd.DataFrame):
    gt = pd.DataFrame(data['평균기온'], columns=['평균기온'])
    X = data.drop('평균기온', axis=1)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA(n_components=0.99)
    pca.fit(X_scaled)
    print(pca.explained_variance_ratio_)
    X_pca = pca.transform(X_scaled)

    columns_pca = [f'PCA_{i}' for i in range(1, X_pca.shape[1] + 1)]
    X_pca_df = pd.DataFrame(X_pca, columns=columns_pca)
    X_pca_df.index = data.index

    df_combined = pd.concat([gt, X_pca_df], axis=1)
    return df_combined

In [ ]:
pca = True
if pca:
    PCA_data = generate_PCA_data(data)

    date_time = pd.DatetimeIndex(data.index)
    day_of_year = date_time.dayofyear

    PCA_data['Day sin'] = np.sin(2 * np.pi * day_of_year / 365)
    PCA_data['Day cos'] = np.cos(2 * np.pi * day_of_year / 365)
    PCA_data.to_csv('../Database/PCA_data.csv')

if not pca:
    date_time = pd.DatetimeIndex(data.index)
    day_of_year = date_time.dayofyear

    data['Day sin'] = np.sin(2 * np.pi * day_of_year / 365)
    data['Day cos'] = np.cos(2 * np.pi * day_of_year / 365)
    data.dropna()
    data.to_csv('../Database/PCA_data.csv')

In [40]:
# fig, ax = plt.subplots(figsize=(10, 8))
# train.boxplot(ax=ax)
# fig.suptitle('Train Data', fontsize=16)
# fig.tight_layout(rect=[0, 0, 1, 0.96])
# fig.show()